### Monte Carlo simulations

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

from src.visualization.plot import plot, track_plots, plot_parameters
ship_data = catalog.load("ship_data")


In [ ]:
regressions = {}
for vmm_name in vmms:
    regressions[vmm_name] = catalog.load(f"{ vmm_name }.motion_regression.joined.regression")

In [ ]:
fig_template = """```{{glue:figure}} {fig_name}
:figwidth: 600px
:name: "{fig_name}"

{caption}
```"""

In [ ]:
print(fig_template.format(fig_name='hej', caption='hej'))

In [ ]:
quantile_cuts_dict={
    'vmm_abkowitz' : [0.6, 0.95],
}

for vmm_name, regression in regressions.items():
    parameters = pd.DataFrame()
    parameters['mean'] = regression.parameters['regressed']
    parameters['std'] = regression.std
    quantile_cuts=quantile_cuts_dict.get(vmm_name,[])
    fig = plot_parameters(parameters, quantile_cuts=quantile_cuts);
    
    fig_name = f"fig_parameters_{vmm_name}"
    glue(fig_name, fig, display=False)
    
    caption = f"Regressed parameters on {vmm_name}"
    print(fig_template.format(fig_name=fig_name, caption=caption))
    
    
    

Hydrodynamic derivatives regressed on all model tests are shown for the examined VMM:s below. The Abkowitz model has a lot of parameters as seen in {numref}`fig_parameters_vmm_abkowitz`. According to the Centrall limit theorem the hydrodynamic derivatives from the OLS regression are Gausian random variables. The bar plots show the mean values of these random variables together with the standard deviation shown as the black error bars. It can be seen that the black error bars are much larger for the Abkowitz model, which means that the parameters of this model are much more uncertain.

```{glue:figure} fig_parameters_vmm_abkowitz
:figwidth: 600px
:name: "fig_parameters_vmm_abkowitz"

Regressed parameters on vmm_abkowitz
```
```{glue:figure} fig_parameters_vmm_martin
:figwidth: 600px
:name: "fig_parameters_vmm_martin"

Regressed parameters on vmm_martin
```
```{glue:figure} fig_parameters_vmm_linear
:figwidth: 600px
:name: "fig_parameters_vmm_linear"

Regressed parameters on vmm_linear
```
```{glue:figure} fig_parameters_vmm_martins_simple
:figwidth: 600px
:name: "fig_parameters_vmm_martins_simple"

Regressed parameters on vmm_martins_simple
```